In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
import os
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
# import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-5/Mask_ERF_iris-5.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [3]:
track = pd.read_csv('../saved-files/threshold-5/final-tracks-threshold-5-excluding-area.csv')
track.head()

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0      4   28  201.219736  300.689233   11                5      423   
1      5   38  202.273236  301.660422   35                5      527   
2      6   37  203.303419  302.324596   49                5      627   
3      4   37  230.460885  211.799724  187                5      432   
4      5   44  231.873681  213.475405  183                5      533   

                  time              timestr  projection_y_coordinate  \
0  2020-03-13 04:20:00  2020-03-13 04:20:00            -2.261220e+06   
1  2020-03-13 04:25:00  2020-03-13 04:25:00            -2.262273e+06   
2  2020-03-13 04:30:00  2020-03-13 04:30:00            -2.263303e+06   
3  2020-03-13 04:20:00  2020-03-13 04:20:00            -2.290461e+06   
4  2020-03-13 04:25:00  2020-03-13 04:25:00            -2.291874e+06   

   projection_x_coordinate   latitude  longitude  cell        time_cell  \
0            217689.233099  69.544704  15.498959   416  0 days 00:00:00   
1            218660.421950  69.534304  15.520784   416  0 days 00:05:00   
2            219324.595821  69.524381  15.534945   416  0 days 00:10:00   
3            128799.724015  69.340584  13.218530   425  0 days 00:00:00   
4            130475.405470  69.326868  13.258308   425  0 days 00:05:00   

     distance   mean dBZ    max dBZ  
0   39.132921  12.845599  38.092506  
1   37.704661  13.138674  36.584485  
2   36.472873  13.454442  38.092506  
3  110.156057  12.845599  38.092506  
4  108.487906  13.138674  36.584485

In [4]:
len(track)

6160

## _CALCULATE AREA_

In [6]:
area = tobac.calculate_area(track, Mask_ERF, method_area=None)
# area1 = (c['area']/1000000)
area1 = np.divide(area['area'],1000000)
area['area($km^2$)'] = area1

In [7]:
track['area($km^2$)'].max()

5516.557463831308

In [8]:
# # finding the maximum area
# z=track[track['area($km^2$)'] == 1433.9665540745268]
# z

In [9]:
tracks = track.drop(columns=['area'])
tracks

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0         4   28  201.219736  300.689233   11                5      423   
1         5   38  202.273236  301.660422   35                5      527   
2         6   37  203.303419  302.324596   49                5      627   
3         4   37  230.460885  211.799724  187                5      432   
4         5   44  231.873681  213.475405  183                5      533   
...     ...  ...         ...         ...  ...              ...      ...   
6155    299   27  271.955203  288.680268  159                5    23039   
6156    300   30  272.843216  289.783693  142                5    23077   
6157    298   34  296.371715  217.621195   30                5    23006   
6158    299   30  296.984193  220.121651   26                5    23042   
6159    300   34  298.121546  222.140438   35                5    23081   

                     time              timestr  projection_y_coordinate  \
0     2020-03-13 04:20:00  2020-03-13 04:20:00            -2.261220e+06   
1     2020-03-13 04:25:00  2020-03-13 04:25:00            -2.262273e+06   
2     2020-03-13 04:30:00  2020-03-13 04:30:00            -2.263303e+06   
3     2020-03-13 04:20:00  2020-03-13 04:20:00            -2.290461e+06   
4     2020-03-13 04:25:00  2020-03-13 04:25:00            -2.291874e+06   
...                   ...                  ...                      ...   
6155  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.331955e+06   
6156  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.332843e+06   
6157  2020-03-14 04:50:00  2020-03-14 04:50:00            -2.356372e+06   
6158  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.356984e+06   
6159  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.358122e+06   

      projection_x_coordinate   latitude  longitude   cell        time_cell  \
0               217689.233099  69.544704  15.498959    416  0 days 00:00:00   
1               218660.421950  69.534304  15.520784    416  0 days 00:05:00   
2               219324.595821  69.524381  15.534945    416  0 days 00:10:00   
3               128799.724015  69.340584  13.218530    425  0 days 00:00:00   
4               130475.405470  69.326868  13.258308    425  0 days 00:05:00   
...                       ...        ...        ...    ...              ...   
6155            205680.267926  68.912816  15.040489  16225  0 days 00:05:00   
6156            206783.692918  68.903864  15.065471  16225  0 days 00:10:00   
6157            134621.195276  68.737655  13.269794  16228  0 days 00:00:00   
6158            137121.651293  68.730769  13.329528  16228  0 days 00:05:00   
6159            139140.437662  68.719331  13.376810  16228  0 days 00:10:00   

        distance   mean dBZ    max dBZ  area($km^2$)  
0      39.132921  12.845599  38.092506     50.741650  
1      37.704661  13.138674  36.584485     49.744965  
2      36.472873  13.454442  38.092506     31.837476  
3     110.156057  12.845599  38.092506    360.167853  
4     108.487906  13.138674  36.584485    362.160182  
...          ...        ...        ...           ...  
6155   52.916488  15.240493  41.116443    452.731337  
6156   52.910819  14.702386  40.106802    453.726574  
6157  122.630657  15.301029  39.603226    161.199380  
6158  120.888777  15.240493  41.116443     75.621784  
6159  119.862226  14.702386  40.106802    157.215939  

[6160 rows x 19 columns]

In [10]:
# all tracks
tracks.to_csv('../saved-files/threshold-5/final-tracks-threshold-5.csv', index=False)

# END 